# Group 12

The youtube video presentation: https://youtu.be/9Rx7i3e21ks

# Question 3 
Implement a model to return the prediction required by the competition. Students are free to reuse
existing code available in open access Spark libraries. This step should also provide some quanti-
tative assessment of the quality of the set of features returned in the previous step. The learning
procedure must be detailed in the notebook. The text should justify the choice of this procedure,
assess its accuracy with respect to the one developed in the point 2 and discuss the results. The use
of figures, formulas, tables and pseudo-code to describe the combination of this novel procedure is
strongly encouraged. Note one third of the score will be attributed on the basis of the quality of the
documentation. On the basis of the described procedure, the team must compute the predictions for
the competition and submit them via the RecSys website.

-------
<p>
It should be noted that due we have sometimes work with samples of data that are less than 1000 to make sure we can easily test our code and analyse the result
</p>

-----------

# Import packages 

In [1]:
import os 

# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import pandas as pd
import time
import numpy as np
import scipy.stats

In [2]:
#This is needed to start a Spark session from the notebook
import os 
os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=4g  pyspark-shell"

from pyspark.sql import SparkSession

In [3]:
#spark.stop()

In [4]:
#os.environ['PYSPARK_SUBMIT_ARGS'] = "--conf spark.driver.memory=2g  pyspark-shell"
#http://localhost:4040/jobs/

from pyspark.sql import SparkSession

# Start Spark session with local master and 2 cores
spark = SparkSession \
    .builder \
    .master("local[3]") \
    .appName("q3_spark_3") \
    .getOrCreate()
###############
#ON THE CLUSTER OF ULB
#spark = SparkSession \
 #   .builder \
  #  .master("local[2]") \
   # .appName("AppName_ul") \
    #.config('spark.ui.port', '4050')\
    #.getOrCreate()
#####################
# Let us retrieve the sparkContext object
sc = spark.sparkContext

22/06/03 21:28:53 WARN Utils: Your hostname, steve-Lenovo-ideapad-310-15ABR resolves to a loopback address: 127.0.1.1; using 192.168.2.11 instead (on interface wlp1s0)
22/06/03 21:28:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/06/03 21:28:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
#!pip install pyspark_dist_explore

In [6]:
#to add new column with default value
from pyspark.sql.functions import lit

from pyspark.sql.functions import col

#import the count function
from pyspark.sql.functions import count
# Import pyspark.sql.functions as F
import pyspark.sql.functions as F
import pyspark.sql as SQL
#https://github.com/Bergvca/pyspark_dist_explore
from pyspark_dist_explore import hist
import matplotlib.pyplot as plt

from pyspark.sql.types import IntegerType
from pyspark.sql.types import StringType
from pyspark.sql.types import DoubleType, ArrayType
from pyspark.sql.types import FloatType
from pyspark.sql.types import TimestampType
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

from pyspark.sql.window import Window as W
from pyspark.ml.classification import RandomForestClassifier
from pyspark.mllib.evaluation import MulticlassMetrics
from sklearn.metrics import classification_report, confusion_matrix
 
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

# Work on the data provided from Q1 and Q2. 

In [7]:
#feature_engineered
x_df = spark.read.load('./X_update.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')
#The items that were viewed in a session.
y_df = spark.read.load('./Y.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')
#final_test.csv
final_test_df = spark.read.load('./final_test.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')
#The candidate items to recommend from. This is one list for both the validation and test set.
candidate_items = spark.read.load('./candidate_items.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')
#leaderboard_test.csv
leaderboard_test_df = spark.read.load('./leaderboard_test.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')
#result_cross_validation.csv
result_cross_validation_df = spark.read.load('./result_cross_validation.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

feature_score_mrmr_df = spark.read.load('./feature_score_mrmr.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')


In [8]:
print(len(x_df.columns), x_df.columns)

83 ['session_id', 'item_id', 'number_of_sessions', 'number_of_item_per_session', 'number_of_items', 'number_of_session_per_item', 'number_of_categories', 'number_of_values', 'number_of_categories_for_session', 'number_of_values_for_session', 'category_id: 1', 'category_id: 2', 'category_id: 3', 'category_id: 4', 'category_id: 5', 'category_id: 6', 'category_id: 7', 'category_id: 8', 'category_id: 9', 'category_id: 10', 'category_id: 11', 'category_id: 12', 'category_id: 13', 'category_id: 14', 'category_id: 15', 'category_id: 16', 'category_id: 17', 'category_id: 18', 'category_id: 19', 'category_id: 20', 'category_id: 21', 'category_id: 22', 'category_id: 23', 'category_id: 24', 'category_id: 25', 'category_id: 26', 'category_id: 27', 'category_id: 28', 'category_id: 29', 'category_id: 30', 'category_id: 31', 'category_id: 32', 'category_id: 33', 'category_id: 34', 'category_id: 35', 'category_id: 36', 'category_id: 37', 'category_id: 38', 'category_id: 39', 'category_id: 40', 'catego

In [9]:
print(len(x_df.columns), final_test_df.columns)

83 ['session_id', 'item_id', 'number_of_sessions', 'number_of_item_per_session', 'number_of_items', 'number_of_session_per_item', 'number_of_categories', 'number_of_values', 'number_of_categories_for_session', 'number_of_values_for_session', 'category_id: 1', 'category_id: 2', 'category_id: 3', 'category_id: 4', 'category_id: 5', 'category_id: 6', 'category_id: 7', 'category_id: 8', 'category_id: 9', 'category_id: 10', 'category_id: 11', 'category_id: 12', 'category_id: 13', 'category_id: 14', 'category_id: 15', 'category_id: 16', 'category_id: 17', 'category_id: 18', 'category_id: 19', 'category_id: 20', 'category_id: 21', 'category_id: 22', 'category_id: 23', 'category_id: 24', 'category_id: 25', 'category_id: 26', 'category_id: 27', 'category_id: 28', 'category_id: 29', 'category_id: 30', 'category_id: 31', 'category_id: 32', 'category_id: 33', 'category_id: 34', 'category_id: 35', 'category_id: 36', 'category_id: 37', 'category_id: 38', 'category_id: 39', 'category_id: 40', 'catego

In [10]:
print((x_df.count(), len(x_df.columns)))

(5743820, 83)


In [11]:
feature_score_mrmr_df.show(1)

+---+--------------------+------+---------------+
|_c0|              scores|col_id|       col_name|
+---+--------------------+------+---------------+
|  0|0.046080561980858796|    32|category_id: 24|
+---+--------------------+------+---------------+
only showing top 1 row



22/06/03 21:30:41 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , scores, col_id, col_name
 Schema: _c0, scores, col_id, col_name
Expected: _c0 but found: 
CSV file: file:///home/steve/Documents/COURSES/PROJECTS/Big%20data%20Distributed%20system-project/doc/feature_score_mrmr.csv


## Get the list of the most ranked columns

In [12]:
feature_score_mrmr_list = feature_score_mrmr_df.select("col_name").rdd.flatMap(lambda x: x).collect()

print (feature_score_mrmr_list)

['category_id: 24', 'number_of_sessions', 'category_id: 16', 'category_id: 70', 'category_id: 51', 'category_id: 20', 'category_id: 54', 'category_id: 22', 'category_id: 48', 'category_id: 12', 'category_id: 6', 'category_id: 11', 'category_id: 40', 'category_id: 10', 'category_id: 72', 'category_id: 14', 'number_of_categories_for_session', 'category_id: 44', 'category_id: 56', 'category_id: 1', 'category_id: 21', 'category_id: 37', 'category_id: 60', 'category_id: 18', 'category_id: 61', 'category_id: 29', 'number_of_item_per_session', 'category_id: 66', 'category_id: 57', 'category_id: 53', 'category_id: 30', 'category_id: 55', 'category_id: 50', 'category_id: 28', 'category_id: 59', 'number_of_values_for_session', 'category_id: 38', 'category_id: 68', 'item_id', 'category_id: 41', 'number_of_items', 'category_id: 8', 'category_id: 33', 'category_id: 42', 'category_id: 34', 'number_of_session_per_item', 'category_id: 25', 'category_id: 62', 'category_id: 7', 'category_id: 19', 'categ

## Joined the X and Y data

In [13]:

#We select the first 10000 to work with the data
limit_val = 1000
df_features = x_df.limit(limit_val)
df_label = y_df.select('purchased').limit(limit_val)
index_list = list(range(1, 1+df_label.count() ))

In [14]:
###### Add an index column to the spark dataframe

#### For the label_df
#convert list to a dataframe

def build_joined_data_from_x_y(dataframeX, dataframeY):
    index_df = sqlContext.createDataFrame([(l,) for l in index_list], ['index'])
    dataframeY = dataframeY.withColumn("idx", F.monotonically_increasing_id())
    dataframeX = dataframeX.withColumn("idx", F.monotonically_increasing_id())
    index_df = index_df.withColumn("idx", F.monotonically_increasing_id())

    windowSpec = W.orderBy("idx")
    dataframeY = dataframeY.withColumn("idx", F.row_number().over(windowSpec))
    dataframeX = dataframeX.withColumn("idx", F.row_number().over(windowSpec))
    index_df = index_df.withColumn("idx", F.row_number().over(windowSpec))
    
    #dataframeY = dataframeY.withColumnRenamed('idx', 'idx2')
    joined_data = dataframeX.join(dataframeY,
              dataframeX["idx"] ==  dataframeY["idx"] 
                                  ,"inner")
    joined_data = joined_data.drop("idx")
    return joined_data


In [15]:
joined_data = build_joined_data_from_x_y(df_features, df_label)
print(joined_data.columns)

['session_id', 'item_id', 'number_of_sessions', 'number_of_item_per_session', 'number_of_items', 'number_of_session_per_item', 'number_of_categories', 'number_of_values', 'number_of_categories_for_session', 'number_of_values_for_session', 'category_id: 1', 'category_id: 2', 'category_id: 3', 'category_id: 4', 'category_id: 5', 'category_id: 6', 'category_id: 7', 'category_id: 8', 'category_id: 9', 'category_id: 10', 'category_id: 11', 'category_id: 12', 'category_id: 13', 'category_id: 14', 'category_id: 15', 'category_id: 16', 'category_id: 17', 'category_id: 18', 'category_id: 19', 'category_id: 20', 'category_id: 21', 'category_id: 22', 'category_id: 23', 'category_id: 24', 'category_id: 25', 'category_id: 26', 'category_id: 27', 'category_id: 28', 'category_id: 29', 'category_id: 30', 'category_id: 31', 'category_id: 32', 'category_id: 33', 'category_id: 34', 'category_id: 35', 'category_id: 36', 'category_id: 37', 'category_id: 38', 'category_id: 39', 'category_id: 40', 'category_

In [16]:
def return_joined_df_session_id(sessionid, joined_df):
    temp_df = joined_df.filter(joined_df.session_id == sessionid)
    return temp_df

In [17]:
def return_x_and_y_df_session_id(sessionid, joined_df):
    temp_df = return_joined_df_session_id(sessionid, joined_df)
    cols = joined_df.columns
    cols.remove('session_id')
    cols.remove('purchased')
    xtemp_df = temp_df.select(cols)
    ytemp_df = temp_df.select('purchased')
    return xtemp_df, ytemp_df

In [18]:
joined_data.limit(10).select('session_id', 'purchased').distinct().show(20)  

22/06/03 21:30:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/03 21:30:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+---------+
|session_id|purchased|
+----------+---------+
|     11897|        0|
|    517715|        0|
|    630526|        0|
|    693471|        1|
|    922323|        1|
|   1059784|        0|
|   1139237|        0|
|   1535716|        0|
|   1558237|        0|
|   1593430|        1|
+----------+---------+



# Apply Random forest on the training set

In [19]:
#https://stackoverflow.com/questions/46710934/pyspark-sql-utils-illegalargumentexception-ufield-features-does-not-exist
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

def construct_ml_features_col(feat_cols_list, featuresDF):
    if 'index' in feat_cols_list:
        feat_cols_list.remove('index')
    assembler = VectorAssembler(
        inputCols=feat_cols_list,
        outputCol='features')
    dataset = assembler.transform(featuresDF)
    #print ('inside construct_ml_features_col **')
    #print (dataset.select('features').show(1))
    return dataset

def construct_join_ml_df(feat_cols_list, featuresDF, labelDF):
    dataset = construct_ml_features_col(feat_cols_list, featuresDF)
    newdata = labelDF.withColumnRenamed('purchased', 'label')
    dataset =  build_joined_data_from_x_y(dataset, newdata)
    dataset = dataset.select(col('label'), col('features'))
    #print ('inside F2 *** construct_join_ml_df **')
    #print(dataset.show(1))
    #newdata.join(dataset, newdata.index == dataset.index).drop("index").select(col('label'), col('features'))
    return dataset

### Define the function to train the model for a given session_id

In [20]:
 
def train_df_model_per_session(session_id, joinedData, feat_score_mrmr_df, number_of_features = 10):
    data = []
     ## Set the number of columns to be used from the ranking algorithm
    df_features_cols_2 = feat_score_mrmr_df.select("col_name").limit(number_of_features).rdd.flatMap(lambda x: x).collect()
    if not ('item_id' in df_features_cols_2):
        df_features_cols_2.append('item_id')
        
     #get the joined DF for the session Id
    joinedDF = return_joined_df_session_id(session_id, joinedData)
    
    #sample the validation data set
    df_sample_validation = joinedDF.sample(withReplacement=False, fraction=0.3, seed=42)
    
    #sample the training data set 
    df_sample_training = joinedDF.sample(withReplacement=False, fraction=0.9, seed=42)
    
    #extract x and y
    df_sample_trainingx,  df_sample_trainingy= return_x_and_y_df_session_id(session_id, df_sample_training)
    df_sample_validationx,  df_sample_validationy= return_x_and_y_df_session_id(session_id, df_sample_validation)
    
   
    #df_features_cols_2.drop
  
    #df_features_cols_2 = list(df_sample_trainingx.columns)
    
    #df_sample_trainingx = df_sample_trainingx.withColumnRenamed('purchased', 'label')
    #print(' before dat_training****' )
    #print (df_sample_trainingx.show(1))
    #dat_training = construct_join_ml_df(df_features_cols_2,df_sample_trainingx, df_sample_trainingy)
    #########################\
    
    df_sample_training = construct_ml_features_col(df_features_cols_2, df_sample_training)
    dat_training = df_sample_training.withColumnRenamed('purchased', 'label')
    dat_training = dat_training.select(col('label'), col('features'))
    ###################################
    #print('sample dattraining = ', dat_training.count() )
    #print(dat_training.show(2))
        
    rf = RandomForestClassifier( labelCol="label", featuresCol="features", seed=0 )
    rf = rf.fit(dat_training)
   
    #######
    #print('data for prediction')
    #print(df_sample_validationx.show(1))
    
    predict_data =  construct_ml_features_col(df_features_cols_2, df_sample_validationx).select(col('features'))
    #print('after tranformation data for prediction')
    #print(predict_data.show(1))
    #####
    pred = rf.transform(predict_data)

    ###
    #print('predict_data ', predict_data.count())
    #print(df_sample_validationx.show(1))
    #print(predict_data.show(1))
    ###
    array_value = np.array(df_sample_validationy.select(col('purchased')).collect() )
    array_prediction_2 = np.array(pred.select(col('prediction')).collect())
    val_2 = np.mean( np.power( array_prediction_2 - array_value  , 2 ) )

    data.append( (session_id, val_2.item()  ) )
        #print("#Features: ", (j+1), X_tr.columns[0:(j+1)], " ; CV error LR =", CV_err[j][i] , " ; CV error DT =", CV_err_2[j][i] )

    columns = ['session_id' ,"mse" ]
    rdd = spark.sparkContext.parallelize(data)
    result_cv = spark.createDataFrame(rdd).toDF(*columns)
    #print(pred.printSchema()) 
    pred2 = pred.select( split_array_to_list(F.col("probability")).alias("prob"))\
                            .select([F.col("prob")[i] for i in range(2)])
    
    pred = build_joined_data_from_x_y(pred, pred2)
    
    ##confusion matrix
    #select only prediction and label columns
    y_true = array_value#pred.select(['label']).collect()
    y_pred = pred.select(['prediction']).collect()
    
    #print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))
    #metrics_rf =  [classification_report(y_true, y_pred), confusion_matrix(y_true, y_pred)]

    return pred, result_cv, rf, df_features_cols_2#, metrics_rf
    

In [21]:
def split_array_to_list(col):
    def to_list(v):
        return v.toArray().tolist()
    return F.udf(to_list, ArrayType(DoubleType()))(col)

In [22]:
pred_rf, result_rf, rf_model, features_list = train_df_model_per_session(922323, joined_data.limit(1000), feature_score_mrmr_df, number_of_features=1 )
print(pred_rf.show())
result_rf.show()

22/06/03 21:30:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/03 21:30:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/03 21:31:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/03 21:31:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/03 21:31:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/03 21:31:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/03 2

[]


22/06/03 21:31:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/03 21:31:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/03 21:31:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/03 21:31:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/03 21:31:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/03 21:31:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------+-------------+-----------+----------+-------+-------+
|features|rawPrediction|probability|prediction|prob[0]|prob[1]|
+--------+-------------+-----------+----------+-------+-------+
+--------+-------------+-----------+----------+-------+-------+

None
+----------+---+
|session_id|mse|
+----------+---+
|    922323|NaN|
+----------+---+



In [23]:
final_test_df.select('session_id', 'item_id').limit(100).show(4)

+----------+-------+
|session_id|item_id|
+----------+-------+
|    488622|      3|
|     81892|      9|
|     92071|      9|
|    257553|      9|
+----------+-------+
only showing top 4 rows



In [24]:
def predict_from_final_session_id(sessionID, final_test_df, rf_model, features_list, k_limit=100):
    jd = return_joined_df_session_id(sessionID, final_test_df.limit(k_limit))
    predict_data =  construct_ml_features_col(features_list, jd).select(col('features'))
    pred2 = rf_model.transform(predict_data)
    pred_t= pred2.select( split_array_to_list(F.col("probability")).alias("prob"))\
                                .select([F.col("prob")[i] for i in range(2)])

    pred2 = build_joined_data_from_x_y(pred2, pred_t)
    tempdf = final_test_df.select('session_id', 'item_id').filter(final_test_df.session_id == sessionID)
    pred2 = build_joined_data_from_x_y(tempdf, pred2)
    return pred2

sessionID= 488622
predict_from_final_session_id(sessionID, final_test_df, rf_model, features_list).show()
#return a df with column session_id|item_id| features|rawPrediction|probability|prediction|prob[0]|prob[1]

22/06/03 21:31:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/03 21:31:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/03 21:31:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/03 21:31:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+-------+---------+-------------+-----------+----------+-------+-------+
|session_id|item_id| features|rawPrediction|probability|prediction|prob[0]|prob[1]|
+----------+-------+---------+-------------+-----------+----------+-------+-------+
|    488622|      3|[0.0,3.0]|   [0.0,11.0]|  [0.0,1.0]|       1.0|    0.0|    1.0|
+----------+-------+---------+-------------+-----------+----------+-------+-------+



### Define the global function which takes a joined dataframe (the training dataset) and train the model

### Define the function to test the trained model on the test set

# Apply random forest on the final test set

# Define the function which predict the first 100 items of a given session

Here are the steps to produce the 100 predictions for a session:
* build a matrix having as column the item id from the candidate_items csv files and the most 10 ranked columns from the MRMR ranking alogrithm (from question 02).
* Train the model from the set of training data for only a sigle session id
* apply the model on the newly formed matrix of the candidate_items and the most relevant MRMR columns
* order the resulting dataframe from descending order (highest to lowest) the probability column and take the first 100
* add a ranking column 
* drop the probability column and return the dataframe having the following column session_id,item_id,rank

<i>NB:</i>
In fact, from the project instruction, the candidate item has the data of all the item_id that have been bought, and that is why we use this data for the prediction

use the final test data set

In [25]:
def return_item_id_mrmr_df(x_df, feature_score_mrmr_df, item_id, k_columns=5):
    
    list_column = feature_score_mrmr_df.select("col_name").limit(k_columns).rdd.flatMap(lambda x: x).collect()
    if not ('item_id' in list_column):
        list_column.append('item_id')
    new_item = x_df.select(list_column)
    new_candidate_item = new_item.filter(new_item.item_id == item_id)
    #new_candidate_item = new_candidate_item.withColumnRenamed('item_id', 'item_id2')
    
    return new_candidate_item

In [26]:
#feature_score_mrmr_list = feature_score_mrmr_df.select("col_name").rdd.flatMap(lambda x: x).collect()
def return_k_bought_itemper_session(session_id, joinedData, feature_score_mrmr_df,  final_test_df, number_of_features=10, value_of_k=2):
    pred_rf, result_rf, rf = train_df_model_per_session(session_id, joined_data, feature_score_mrmr_df, number_of_features)
    print(pred_rf.show())
    result_rf.show()
    
    ## work on the candidate item
    candidate_item_list = candidate_items.select("item_id").limit(value_of_k).rdd.flatMap(lambda x: x).collect()
    df_features_cols_2 = feature_score_mrmr_df.select("col_name").limit(number_of_features).rdd.flatMap(lambda x: x).collect()
    for item_id in candidate_item_list:
        item_id_df =  return_item_id_mrmr_df(final_test_df, feature_score_mrmr_df, item_id=4, k_columns=5)
        predict_data =  construct_ml_features_col(df_features_cols_2, item_id_df).select(col('features'))
        pred = rf.transform(predict_data)
        print(pred)
    
    
    

# Define the general function which work for all the session id present